In [ ]:
#install numpy、pandas
import numpy as np
import pandas as pd


In [ ]:
#載入資料
df_all = pd.read_csv('bank/bank-full.csv', sep=';')

#將項目名稱改成中文，確認資料框內容使用display(df_all.head())
columns = [
    '年齡', '職業', '婚姻', '學歷', '違約', '平均餘額', '房屋貸款', '個人信貸', '聯絡方式', '最近一次通話日期', '最近一次通話月份', '最近一次通話秒數', 
    '通話次數_促銷期間', '上次促銷後_經過天數', '通話次數_促銷之前', '上次促銷結果', '本次促銷結果'
]
df_all.columns = columns

#確認訓練資料的比數與項目數
print(df_all.shape)

#確認本次促銷結果的分布、成功率，計算結果次數用.value_counts()
print(df_all['本次促銷結果'].value_counts())
rate = df_all['本次促銷結果'].value_counts()['yes']/len(df_all)
print(f'銷售成功率: {rate:.4f}')


(45211, 17)
本次促銷結果
no     39922
yes     5289
Name: count, dtype: int64
銷售成功率: 0.1170


In [ ]:
#確認缺失值
print(df_all.isnull().sum())

年齡            0
職業            0
婚姻            0
學歷            0
違約            0
平均餘額          0
房屋貸款          0
個人信貸          0
聯絡方式          0
最近一次通話日期      0
最近一次通話月份      0
最近一次通話秒數      0
通話次數_促銷期間     0
上次促銷後_經過天數    0
通話次數_促銷之前     0
上次促銷結果        0
本次促銷結果        0
dtype: int64


concat()說明書
pandas.concat(objs, axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=False, copy=True)
objs：一個可迭代物件（list、tuple）包含要連接的 DataFrame 或 Series。
axis：連接軸，0 表示縱向（行連接，預設），1 表示橫向（列連接）。
join：如何處理索引的聯集。'outer'（預設）為聯集，'inner'為交集。
ignore_index：是否重新索引結果，若True，結果索引將會從0開始。
keys：為連接的資料添加層次化索引標籤。
verify_integrity：是否檢查索引有無重複。
sort：是否排序非連接軸的索引。

資料處理Label encoding或One hot encoding
get_dummies()說明書
pd.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, drop_first=False, dtype=None)
data：欲處理的DataFrame或Series。
columns：選擇指定的欄位進行轉換，不指定則會對所有類別型欄位處理。
prefix：轉換後欄位名前綴，預設用原本欄位名。
prefix_sep：前綴與欄位值間隔符號（預設為 _）。
drop_first：是否刪除第一個類別，避免虛擬變數陷阱（可用於避免多重共線性）。
dummy_na：是否當成一類別值對待空值。
dtype：轉換後的資料型態。

In [ ]:
#使用grt_dummies函式對種類值進行One-Hot編碼
#把 DataFrame df 中名為 column 的**類別欄（categorical）**做 one-hot encoding（虛擬變數化）
def enc(df, column):
    df_dummy = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df.drop([column],axis=1), df_dummy],axis=1)
    return df

